# SSII - Práctica Final: Recomendador de películas con MovieLens

## Grupo 1: Ignacio Gil, Álvaro Farreny y Carlos González

In [39]:
""" 
pip install pandas
pip install numpy
python -m pip install -U matplotlib
pip install seaborn
python -m pip install requests
pip install -U scikit-learn   
pip install beautifulsoup4
"""

import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline
import seaborn as sns
import requests as rq
import time
import math
import datetime

from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.metrics import confusion_matrix

from bs4 import BeautifulSoup as bs

In [40]:
# Leemos nuestros dataframes
movies = pd.read_csv("./ml-latest-small/movies.csv", sep=",")
ratings = pd.read_csv("./ml-latest-small/ratings.csv", sep=",")
tags = pd.read_csv("./ml-latest-small/tags.csv", sep=",")
links = pd.read_csv("./ml-latest-small/links.csv", sep=",")

# Eliminamos nuelos
movies.dropna(inplace=True)
ratings.dropna(inplace=True)
tags.dropna(inplace=True)
links.dropna(inplace=True)

In [41]:
# Extraemos el año del título
movies['year'] = movies.title.str.extract("\((\d{4})\)", expand=True)
movies.year = pd.to_datetime(movies.year, format='%Y')
movies.year = movies.year.dt.year
movies.title = movies.title.str[:-7]

In [42]:
'''# Separamos los géneros
movies['genres'] = movies['genres'].str.split('|')
dfx = pd.get_dummies(pd.DataFrame(movies['genres'].tolist()).stack()).sum(level=0)
movies = pd.concat([movies, dfx], axis=1).drop(columns=['genres'])

generos = movies[['movieId', '(no genres listed)', 'Action', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']].copy()'''

"# Separamos los géneros\nmovies['genres'] = movies['genres'].str.split('|')\ndfx = pd.get_dummies(pd.DataFrame(movies['genres'].tolist()).stack()).sum(level=0)\nmovies = pd.concat([movies, dfx], axis=1).drop(columns=['genres'])\n\ngeneros = movies[['movieId', '(no genres listed)', 'Action', 'Adventure',\n       'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',\n       'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery',\n       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']].copy()"

In [43]:
movies.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,2,Jumanji,Adventure|Children|Fantasy,1995.0
2,3,Grumpier Old Men,Comedy|Romance,1995.0
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995.0
4,5,Father of the Bride Part II,Comedy,1995.0


In [44]:
# Cambiamos los timestamps de ratings y tags de segundos a años
ratings.timestamp = pd.to_datetime(ratings.timestamp, infer_datetime_format=True)
ratings.timestamp = ratings.timestamp.dt.year

tags.timestamp = pd.to_datetime(tags.timestamp, infer_datetime_format=True)
tags.timestamp = tags.timestamp.dt.year

In [45]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,1970
1,1,3,4.0,1970
2,1,6,4.0,1970
3,1,47,5.0,1970
4,1,50,5.0,1970


In [46]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [48]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1970
1,2,60756,Highly quotable,1970
2,2,60756,will ferrell,1970
3,2,89774,Boxing story,1970
4,2,89774,MMA,1970


In [49]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations

# TF-IDF para las diferentes combinaciones de géneros
tf = TfidfVectorizer(analyzer=lambda s: (c for i in range(1,4)
                     for c in combinations(s.split('|'), r=i)))

# Formamos nuestra matriz con los géneros
matriz = tf.fit_transform(movies.genres)
# Aplicamos a esta matriz la similitud del coseno
similitud = cosine_similarity(matriz)
# Creamos un DF con esta similitud
similitud_df = pd.DataFrame(similitud, index=movies['title'], columns=movies['title'])

In [60]:
similitud_df

title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,...,Gintama: The Movie,anohana: The Flower We Saw That Day - The Movie,Silver Spoon,Love Live! The School Idol Movie,Jon Stewart Has Left the Building,Black Butler: Book of the Atlantic,No Game No Life: Zero,Flint,Bungo Stray Dogs: Dead Apple,Andrew Dice Clay: Dice Rules
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.474735,0.033432,0.019663,0.082550,0.000000,0.033432,0.275655,0.000000,0.038862,...,0.090020,0.084617,0.038306,0.159254,0.0,0.306924,0.487104,0.00000,0.086065,0.082550
Jumanji,0.474735,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.580651,0.000000,0.081861,...,0.000000,0.000000,0.000000,0.000000,0.0,0.060495,0.096008,0.00000,0.000000,0.000000
Grumpier Old Men,0.033432,0.000000,1.000000,0.588129,0.404997,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.043147,0.000000,0.187935,0.000000,0.0,0.043247,0.068635,0.00000,0.000000,0.404997
Waiting to Exhale,0.019663,0.000000,0.588129,1.000000,0.238191,0.000000,0.588129,0.000000,0.000000,0.000000,...,0.025376,0.055954,0.513299,0.000000,0.0,0.025435,0.040366,0.21998,0.000000,0.238191
Father of the Bride Part II,0.082550,0.000000,0.404997,0.238191,1.000000,0.000000,0.404997,0.000000,0.000000,0.000000,...,0.106537,0.000000,0.464039,0.000000,0.0,0.106784,0.169471,0.00000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Black Butler: Book of the Atlantic,0.306924,0.060495,0.043247,0.025435,0.106784,0.040749,0.043247,0.000000,0.146144,0.038802,...,0.393689,0.109457,0.049552,0.206006,0.0,1.000000,0.630100,0.00000,0.381193,0.106784
No Game No Life: Zero,0.487104,0.096008,0.068635,0.040366,0.169471,0.000000,0.068635,0.000000,0.000000,0.000000,...,0.184807,0.173714,0.078641,0.326942,0.0,0.630100,1.000000,0.00000,0.176687,0.169471
Flint,0.000000,0.000000,0.000000,0.219980,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.254359,0.428561,0.000000,0.0,0.000000,0.000000,1.00000,0.000000,0.000000


In [12]:
# Nuestro valor k indicará la cantidad de recomendaciones que damos
def genre_recommendations(i, M, items, k=10):
    """
    i : str
        Movie (index of the similarity dataframe)
    M : pd.DataFrame
        Similarity dataframe, symmetric, with movies as indices and columns
    items : pd.DataFrame
        Contains both the title and some other features used to define similarity
    k : int
        Amount of recommendations to return
    """
    index = M.loc[:,i].to_numpy().argpartition(range(-1,-k,-1))
    closest = M.columns[index[-1:-(k+2):-1]]
    closest = closest.drop(i, errors='ignore')
    return pd.DataFrame(closest).merge(items).head(k)

In [13]:
genre_recommendations('Toy Story', similitud_df, movies[['title', 'genres']])

,title,genres
0,"Adventures of Rocky and Bullwinkle, The",Adventure|Animation|Children|Comedy|Fantasy
1,"Emperor's New Groove, The",Adventure|Animation|Children|Comedy|Fantasy
2,Toy Story 2,Adventure|Animation|Children|Comedy|Fantasy
3,Antz,Adventure|Animation|Children|Comedy|Fantasy
4,Turbo,Adventure|Animation|Children|Comedy|Fantasy
5,"Tale of Despereaux, The",Adventure|Animation|Children|Comedy|Fantasy
6,"Monsters, Inc.",Adventure|Animation|Children|Comedy|Fantasy
7,Shrek the Third,Adventure|Animation|Children|Comedy|Fantasy
8,The Good Dinosaur,Adventure|Animation|Children|Comedy|Fantasy
9,Asterix & Obelix vs. Caesar (Astérix et Obélix...,Adventure|Children|Comedy|Fantasy


## Obtener Similitudes a través de TF-IDF y Similitud del coseno

In [6]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations

# Leemos nuestros dataframes
movies = pd.read_csv("./ml-latest-small/movies.csv", sep=",")
sinopsis1 = pd.read_csv("./ml-latest-small/sinopsis1.csv", sep=",")
sinopsis2 = pd.read_csv("./ml-latest-small/sinopsis2.csv", sep=",")
sinopsis3 = pd.read_csv("./ml-latest-small/sinopsis3.csv", sep=",")
sinopsis4 = pd.read_csv("./ml-latest-small/sinopsis4.csv", sep=",")
sinopsis5 = pd.read_csv("./ml-latest-small/sinopsis5.csv", sep=",")

# Extraemos el año del título
movies['year'] = movies.title.str.extract("\((\d{4})\)", expand=True)
movies.year = pd.to_datetime(movies.year, format='%Y')
movies.year = movies.year.dt.year
movies.title = movies.title.str[:-7]

# TF-IDF para las diferentes combinaciones de géneros
tfG = TfidfVectorizer(analyzer=lambda s: (c for i in range(1,4)
                     for c in combinations(s.split('|'), r=i)))
tfS = TfidfVectorizer(analyzer=lambda s: (c for i in range(1,4)
                     for c in combinations(s.split(' '), r=i)))

# Formamos nuestra matriz con los géneros
matrizG = tfG.fit_transform(movies.genres)
matrizS1 = tfS.fit_transform(sinopsis1.sinopsis)
# Aplicamos a esta matriz la similitud del coseno
similitudG = cosine_similarity(matrizG)
similitudS1 = cosine_similarity(matrizS1)

pd.DataFrame(similitudG).to_csv("similitudG.csv", header=False, index=False)
pd.DataFrame(similitudS1).to_csv("similitudS1.csv", header=False, index=False)

KeyboardInterrupt: 

In [ ]:

matrizS2 = tfS.fit_transform(sinopsis2.sinopsis)
similitudS2 = cosine_similarity(matrizS2)
pd.DataFrame(similitudS1).to_csv("similitudS1.csv", header=False, index=False)